rerun on 2021-12-07 after correcting for salinity of media samples

In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob



os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption')
matplotlib.__version__

'3.3.3'

In [2]:
outdir='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/'

#!head /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv

In [3]:
pd.set_option("max_rows", 10)


In [4]:
#import RSG SAG data 
columns=['name', 'GTDB_classification', 'Predicted_respiration_rate_fmol_O2perhr', 
         'Sample_collection_Date', 'FACS_date', 'gate']
RSG_SAGs=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv',
                    usecols=columns)
RSG_SAGs=RSG_SAGs[RSG_SAGs['Sample_collection_Date']==RSG_SAGs['FACS_date']]
RSG_SAGs['count']=1
RSG_SAGs=RSG_SAGs.rename(columns={'GTDB_classification':'genus', 'Sample_collection_Date':'date'})
RSG_SAGs=RSG_SAGs[RSG_SAGs['gate']=='Respiring cells (RSG-positive)']
RSG_SAGs
avg_RSG=pd.pivot_table(RSG_SAGs, values='Predicted_respiration_rate_fmol_O2perhr', 
                        columns='genus', index='date', aggfunc=np.mean)

avg_RSG=avg_RSG.reset_index()
avg_RSG

genus,date,AAA536-G10,AG-337-I02,AG-414-E02,AG-422-B15,ASP10-02a,Akkermansiaceae,Algibacter_B,Amylibacter,Aquiluna,...,UBA9320,UBA952,UBA955,UBA974,UBA985,UBA9926,Ulvibacter,Unclassified,Winogradskyella,Yoonia
0,170412,0.004427,NaN,0.004464,0.004552,3.188656,0.005161,1.275009,0.137650,NaN,...,0.021816,0.017333,NaN,NaN,0.006618,0.028604,1.10427,5.003559,0.012836,38.227900
1,170818,0.004944,0.010778,NaN,0.005821,NaN,NaN,NaN,0.085607,0.005300,...,NaN,0.015579,NaN,0.011494,NaN,NaN,NaN,0.695980,NaN,NaN
2,171102,0.004879,0.010198,NaN,0.005728,NaN,NaN,NaN,0.478868,NaN,...,NaN,0.085816,NaN,NaN,NaN,0.020229,NaN,25.585987,0.013650,NaN
3,181030,0.004542,0.009881,NaN,NaN,NaN,NaN,NaN,0.070978,0.004544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.053808,NaN,NaN
4,190402,0.004337,NaN,NaN,0.004214,2.069933,NaN,NaN,0.075798,0.005750,...,0.097805,NaN,NaN,NaN,NaN,0.010926,0.12673,0.526985,0.006314,0.595827
5,190709,0.003866,NaN,NaN,NaN,NaN,0.003982,NaN,0.144740,0.007297,...,0.007881,0.005532,0.020497,NaN,NaN,0.008317,NaN,0.018711,0.008133,NaN


In [5]:
#calculate each GEN's percentate at every timepoint
RSG_count_pivot=pd.pivot_table(RSG_SAGs, values='count', 
                        columns='genus', index='date', aggfunc=np.sum)
RSG_count_pivot.sum(axis=1)

RSG_perc=RSG_count_pivot.div(RSG_count_pivot.sum(axis=1), axis=0)
RSG_perc


genus,AAA536-G10,AG-337-I02,AG-414-E02,AG-422-B15,ASP10-02a,Akkermansiaceae,Algibacter_B,Amylibacter,Aquiluna,Arcobacteraceae,...,UBA9320,UBA952,UBA955,UBA974,UBA985,UBA9926,Ulvibacter,Unclassified,Winogradskyella,Yoonia
date,,,,,,,,,,,,,,,,,,,,,
170412,0.003534,NaN,0.003534,0.003534,0.017668,0.007067,0.007067,0.053004,NaN,NaN,...,0.007067,0.003534,NaN,NaN,0.003534,0.010601,0.007067,0.268551,0.003534,0.003534
170818,0.013468,0.003367,NaN,0.006734,NaN,NaN,NaN,0.016835,0.006734,NaN,...,NaN,0.003367,NaN,0.003367,NaN,NaN,NaN,0.259259,NaN,NaN
171102,0.006734,0.003367,NaN,0.003367,NaN,NaN,NaN,0.026936,NaN,NaN,...,NaN,0.006734,NaN,NaN,NaN,0.013468,NaN,0.326599,0.003367,NaN
181030,0.010638,0.014184,NaN,NaN,NaN,NaN,NaN,0.014184,0.003546,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.156028,NaN,NaN
190402,0.018248,NaN,NaN,0.010949,0.014599,NaN,NaN,0.054745,0.003650,0.00365,...,0.003650,NaN,NaN,NaN,NaN,0.007299,0.003650,0.200730,0.007299,0.003650
190709,0.003610,NaN,NaN,NaN,NaN,0.003610,NaN,0.018051,0.014440,NaN,...,0.010830,0.003610,0.00361,NaN,NaN,0.014440,NaN,0.216606,0.003610,NaN


In [6]:
#replace all genera with <3 fresh RSG labeled SAGS and drop genera that are <3 RSG SAGs at all timepoints
threeRSG_SAGs=RSG_count_pivot[RSG_count_pivot >= 3 ].copy()
threeRSG_SAGs=threeRSG_SAGs.dropna(how='all', axis=1)
threeRSG_SAGs=threeRSG_SAGs.iloc[2:]

threeRSG_SAGs


#OLD CODE THAT USED 1% instead of 3 SAGs

#threeRSG_SAGs_perc=RSG_perc.apply(lambda x: [y if y >=0.01 else 0 for y in x])
#threeRSG_SAGs_perc=threeRSG_SAGs_perc.replace(0, np.nan)
#threeRSG_SAGs_perc=threeRSG_SAGs.div(RSG_count_pivot.sum(axis=1), axis=0)
#threeRSG_SAGs_perc
#threeRSG_SAGs_perc=threeRSG_SAGs_perc.reset_index()
#threeRSG_SAGs_perc=threeRSG_SAGs_perc.iloc[2:]
#threeRSG_SAGs_perc

genus,AAA536-G10,AG-337-I02,AG-422-B15,ASP10-02a,Amylibacter,Aquiluna,BACL14,D2472,GCA-002733185,HC6-5,...,Thioglobus,UBA10364,UBA3537,UBA4465,UBA4582,UBA6772,UBA7428,UBA9320,UBA9926,Unclassified
date,,,,,,,,,,,,,,,,,,,,,
171102,NaN,NaN,NaN,NaN,8.0,NaN,NaN,4.0,NaN,NaN,...,8.0,8.0,3.0,NaN,NaN,NaN,NaN,NaN,4.0,97.0
181030,3.0,4.0,NaN,NaN,4.0,NaN,4.0,10.0,NaN,NaN,...,12.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,44.0
190402,5.0,NaN,3.0,4.0,15.0,NaN,NaN,22.0,3.0,6.0,...,6.0,3.0,NaN,3.0,5.0,NaN,7.0,NaN,NaN,55.0
190709,NaN,NaN,NaN,NaN,5.0,4.0,NaN,7.0,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,4.0,3.0,3.0,4.0,60.0


In [7]:
threeRSG_SAGs_perc=threeRSG_SAGs.div(RSG_count_pivot.sum(axis=1), axis=0)
threeRSG_SAGs_perc=threeRSG_SAGs_perc.iloc[2:]
threeRSG_SAGs_perc=threeRSG_SAGs_perc.reset_index()
threeRSG_SAGs_perc

genus,date,AAA536-G10,AG-337-I02,AG-422-B15,ASP10-02a,Amylibacter,Aquiluna,BACL14,D2472,GCA-002733185,...,Thioglobus,UBA10364,UBA3537,UBA4465,UBA4582,UBA6772,UBA7428,UBA9320,UBA9926,Unclassified
0,171102,NaN,NaN,NaN,NaN,0.026936,NaN,NaN,0.013468,NaN,...,0.026936,0.026936,0.010101,NaN,NaN,NaN,NaN,NaN,0.013468,0.326599
1,181030,0.010638,0.014184,NaN,NaN,0.014184,NaN,0.014184,0.035461,NaN,...,0.042553,NaN,NaN,NaN,NaN,NaN,0.014184,NaN,NaN,0.156028
2,190402,0.018248,NaN,0.010949,0.014599,0.054745,NaN,NaN,0.080292,0.010949,...,0.021898,0.010949,NaN,0.010949,0.018248,NaN,0.025547,NaN,NaN,0.200730
3,190709,NaN,NaN,NaN,NaN,0.018051,0.01444,NaN,0.025271,NaN,...,0.025271,NaN,NaN,NaN,NaN,0.01444,0.010830,0.01083,0.014440,0.216606


In [8]:
#Create list of GENs that are >1% for each timepoint
perc_RSG_melt=pd.melt(threeRSG_SAGs_perc, id_vars='date', value_name='relative_abundance')
perc_RSG_melt=perc_RSG_melt.dropna(how='any', axis=0)


df_RSG_GENS_171102=perc_RSG_melt[perc_RSG_melt['date']==171102]
RSG_1perc_GENs_171102=df_RSG_GENS_171102['genus'].values.tolist()

df_RSG_GENS_181030=perc_RSG_melt[perc_RSG_melt['date']==181030]
RSG_1perc_GENs_181030=df_RSG_GENS_181030['genus'].values.tolist()

df_RSG_GENS_190402=perc_RSG_melt[perc_RSG_melt['date']==190402]
RSG_1perc_GENs_190402=df_RSG_GENS_190402['genus'].values.tolist()

df_RSG_GENS_190709=perc_RSG_melt[perc_RSG_melt['date']==190709]
RSG_1perc_GENs_190709=df_RSG_GENS_190709['genus'].values.tolist()

#RSG_1perc_GENs_181030

In [9]:
perc_RSG_melt['date']=perc_RSG_melt['date'].astype(str)


perc_RSG_melt

,date,genus,relative_abundance
1,181030,AAA536-G10,0.010638
2,190402,AAA536-G10,0.018248
5,181030,AG-337-I02,0.014184
10,190402,AG-422-B15,0.010949
14,190402,ASP10-02a,0.014599
...,...,...,...
191,190709,UBA9926,0.014440
192,171102,Unclassified,0.326599
193,181030,Unclassified,0.156028
194,190402,Unclassified,0.200730


In [10]:
avg_RSG_melt=pd.melt(avg_RSG, id_vars='date', value_name='Avg_O2_consumed_per_cell')
avg_RSG_melt['date']=avg_RSG_melt['date'].astype(str)
avg_RSG_melt

,date,genus,Avg_O2_consumed_per_cell
0,170412,AAA536-G10,0.004427
1,170818,AAA536-G10,0.004944
2,171102,AAA536-G10,0.004879
3,181030,AAA536-G10,0.004542
4,190402,AAA536-G10,0.004337
...,...,...,...
1063,170818,Yoonia,NaN
1064,171102,Yoonia,NaN
1065,181030,Yoonia,NaN
1066,190402,Yoonia,0.595827


In [11]:
O2_consumption_3SAG_RSG_Genera=perc_RSG_melt.merge(avg_RSG_melt, how='left', on=['date', 'genus'])
O2_consumption_3SAG_RSG_Genera.to_csv(outdir + 'Average_O2_consumption_for_Genera_with_3+_RSG_SAGs_per_day.csv')

O2_consumption_3SAG_RSG_Genera

,date,genus,relative_abundance,Avg_O2_consumed_per_cell
0,181030,AAA536-G10,0.010638,0.004542
1,190402,AAA536-G10,0.018248,0.004337
2,181030,AG-337-I02,0.014184,0.009881
3,190402,AG-422-B15,0.010949,0.004214
4,190402,ASP10-02a,0.014599,2.069933
...,...,...,...,...
87,190709,UBA9926,0.014440,0.008317
88,171102,Unclassified,0.326599,25.585987
89,181030,Unclassified,0.156028,0.053808
90,190402,Unclassified,0.200730,0.526985
